In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [2]:
#Creating Spark Session (It will executed in the Local Not in the Master Node)
spark=SparkSession.builder.master("local[*]").appName("demo").getOrCreate()
spark

23/03/23 16:28:44 WARN Utils: Your hostname, ZSCHN01LP0253L resolves to a loopback address: 127.0.1.1; using 192.168.229.237 instead (on interface wlp0s20f3)
23/03/23 16:28:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/23 16:28:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/23 16:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/23 16:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/03/23 16:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/03/23 16:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/03/23 16:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/03/23 16:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
23/03/23 16:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
23/03/23 16:28:45 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
23/03/23 16:28:45 WARN Utils: Service 'SparkUI' could not bind on

In [3]:
empDf = spark.read.option("header",True).option("inferSchema",True).csv("../Spark-main/employees.csv")
empDf.printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



In [4]:
from pyspark.sql.functions import *

### Spark DataFrame to SQL Table

In [5]:
#Note : Converting Dataframe to SQL type table
empDf.createOrReplaceTempView("employee")
spark.sql(" select * from employee limit 5").show()

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|  MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|  MK_REP|  6000|            - |       201|           20|
+-----------+---

In [6]:
df = spark.sql("select employee_id,salary from employee")
df.show(100)

+-----------+------+
|employee_id|salary|
+-----------+------+
|        198|  2600|
|        199|  2600|
|        200|  4400|
|        201| 13000|
|        202|  6000|
|        203|  6500|
|        204| 10000|
|        205| 12008|
|        206|  8300|
|        100| 24000|
|        101| 17000|
|        102| 17000|
|        103|  9000|
|        104|  6000|
|        105|  4800|
|        106|  4800|
|        107|  4200|
|        108| 12008|
|        109|  9000|
|        110|  8200|
|        111|  7700|
|        112|  7800|
|        113|  6900|
|        114| 11000|
|        115|  3100|
|        116|  2900|
|        117|  2800|
|        118|  2600|
|        119|  2500|
|        120|  8000|
|        121|  8200|
|        122|  7900|
|        123|  6500|
|        124|  5800|
|        125|  3200|
|        126|  2700|
|        127|  2400|
|        128|  2200|
|        129|  3300|
|        130|  2800|
|        131|  2500|
|        132|  2100|
|        133|  3300|
|        134|  2900|
|        135|

In [7]:
spark.sql("select department_id, sum(salary) as sum_salary from employee group by department_id").show()

+-------------+----------+
|department_id|sum_salary|
+-------------+----------+
|           20|     19000|
|           40|      6500|
|          100|     51608|
|           10|      4400|
|           50|     85600|
|           70|     10000|
|           90|     58000|
|           60|     28800|
|          110|     20308|
|           30|     24900|
+-------------+----------+



In [8]:
spark.sql("select employee_id, department_id, rank() over(partition by department_id order by salary desc) as rank_salary from employee").show()

+-----------+-------------+-----------+
|employee_id|department_id|rank_salary|
+-----------+-------------+-----------+
|        200|           10|          1|
|        201|           20|          1|
|        202|           20|          2|
|        114|           30|          1|
|        115|           30|          2|
|        116|           30|          3|
|        117|           30|          4|
|        118|           30|          5|
|        119|           30|          6|
|        203|           40|          1|
|        121|           50|          1|
|        120|           50|          2|
|        122|           50|          3|
|        123|           50|          4|
|        124|           50|          5|
|        137|           50|          6|
|        129|           50|          7|
|        133|           50|          7|
|        125|           50|          9|
|        138|           50|          9|
+-----------+-------------+-----------+
only showing top 20 rows

